In [1]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import timm
import torchsummary
import imageio
import numpy as np
import matplotlib.pyplot as plt
import os
from torch.cuda.amp import GradScaler, autocast

from tqdm.notebook import tqdm

import sys
sys.path.append('../DataLoader')

from dataloader import SunImageDataset

from torch.func import stack_module_state
from torch.func import vmap

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

cuda


In [3]:
# Hyper-parameters
input_size = 224*224
hidden_size = 166
num_epochs = 100
batch_size = 2
learning_rate = 0.001
# dropout = 0.6990787087509548

In [4]:
dataset = SunImageDataset(csv_file="D:\\dataset.csv", offset=0, transform=transforms.ToTensor())
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
trainset, testset = torch.utils.data.Subset(dataset, range(train_size)), torch.utils.data.Subset(dataset, range(train_size, len(dataset)))

trainloader = torch.utils.data.DataLoader(dataset=trainset, batch_size=batch_size, shuffle=True)

# # Get a batch of training data
# dataiter = iter(trainloader)
# images, labels = next(dataiter)
# images = torch.stack(images)
# print(images.shape)
# print(labels.shape)

# print(images)
# print(labels)

In [ ]:

class GmiSwinTransformer(nn.Module):
    def __init__(self, hidden_size: int):
        super(GmiSwinTransformer, self).__init__()
        
        # Batch normalization for 3 channels
        self.bn = nn.BatchNorm2d(3)
        
        # Initialize Swin Transformer
        self.pretrained_model = timm.create_model(
            'swin_base_patch4_window7_224',
            pretrained=True,
            num_classes=hidden_size
        )
        
        
        # Fully connected layers
        self.fc = nn.Sequential(
            nn.LeakyReLU(),
            nn.Linear(hidden_size*10, hidden_size),
            nn.Dropout(p=0.5),  # Added dropout probability
            nn.LeakyReLU(),
            nn.Linear(hidden_size, 1),
            nn.LeakyReLU()
        )
    
    def forward(self, images) -> torch.Tensor:
        """
        Batch should be in format:
        {
            'images': torch.FloatTensor((10, 1, 224, 224))
        }
        """
        # # Print input shape for debugging
        # # print("Input shape:", images.shape)
        # image_features = torch.zeros(images.shape[0],images.shape[1], hidden_size).to(device)
        # for i in range(images.shape[0]):
        #     image = images[i, :, :, :]
        #     # Pretrained swin transformer accepts three channel images
        #     three_channel = torch.stack([image,image,image], dim=1).squeeze(2)
        #     # print("three_channel", three_channel.size())
        #     # Model learns optimal initial normalisation
        #     normalized_images = self.bn(three_channel)
        #     # Get image features
        #     image_feature = self.pretrained_model.forward(normalized_images)
        #     image_features[i] = image_feature
        # # print("image_features before reshaping", image_features.size())
        # image_features = image_features.view(image_features.size(0), -1)
        # print("image_features after reshaping", image_features.size())
        
        batch_size = images.shape[0]
        
        images = images.reshape(-1, 1, 224, 224)
        images = torch.cat([images, images, images], dim=1)
        normalized_images = self.bn(images)
        features = self.pretrained_model(normalized_images)
        image_features = features.view(batch_size, -1)
        
        output = self.fc(image_features)
        return output

# Initialize model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GmiSwinTransformer(hidden_size=hidden_size).to(device)

print(torchsummary.summary(model, (10, 1, 224, 224)))

# Loss and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

c:\Users\mirzi\miniconda3\envs\gmimodel\lib\site-packages\torch\nn\modules\module.py:1329: UserWarning: expandable_segments not supported on this platform (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\c10/cuda/CUDAAllocatorConfig.h:28.)
  return t.to(


Layer (type:depth-idx)                        Output Shape              Param #
├─BatchNorm2d: 1-1                            [-1, 3, 224, 224]         6
├─SwinTransformer: 1-2                        [-1, 166]                 --
|    └─PatchEmbed: 2-1                        [-1, 56, 56, 128]         --
|    |    └─Conv2d: 3-1                       [-1, 128, 56, 56]         6,272
|    |    └─LayerNorm: 3-2                    [-1, 56, 56, 128]         256
|    └─Sequential: 2-2                        [-1, 7, 7, 1024]          --
|    |    └─SwinTransformerStage: 3-3         [-1, 56, 56, 128]         397,896
|    |    └─SwinTransformerStage: 3-4         [-1, 28, 28, 256]         1,714,320
|    |    └─SwinTransformerStage: 3-5         [-1, 14, 14, 512]         57,317,920
|    |    └─SwinTransformerStage: 3-6         [-1, 7, 7, 1024]          27,304,512
|    └─LayerNorm: 2-3                         [-1, 7, 7, 1024]          2,048
|    └─ClassifierHead: 2-4                    [-1, 166]      

In [6]:
torch.cuda.empty_cache()

In [8]:
# Training the model
n_total_steps = len(trainloader)
for epoch in range(num_epochs):
    train_losses = []
    
    for i, (images, labels) in tqdm(enumerate(trainloader), desc="Training Progress", total=len(trainloader)):
        # Move images and labels to device
        images = torch.stack(images).to(device).float()
        # print("images before reshaping", images.size())
        images = images.permute(1, 0, 2, 3, 4)  # Change shape to [5, 10, 1, 224, 224]
        # print("images after reshaping", images.size())
        labels = labels.to(device).float()

        # Forward pass with autograd
        outputs = model(images).squeeze(1)
        loss = criterion(outputs, labels)
        print("Loss", loss)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # Store the loss
        train_losses.append(loss.item())
    print(f'Epoch [{epoch+1}], Loss: {loss.item():.4f}')
    plt.plot(train_losses, label='Training loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()
    

Training Progress:   0%|          | 0/409 [00:00<?, ?it/s]

Loss tensor(13.1562, device='cuda:0', grad_fn=<MseLossBackward0>)
Loss tensor(3.6909, device='cuda:0', grad_fn=<MseLossBackward0>)
Loss tensor(21.9439, device='cuda:0', grad_fn=<MseLossBackward0>)
Loss tensor(6.3393, device='cuda:0', grad_fn=<MseLossBackward0>)
Loss tensor(9.5344, device='cuda:0', grad_fn=<MseLossBackward0>)
Loss tensor(21.7177, device='cuda:0', grad_fn=<MseLossBackward0>)
Loss tensor(10.7297, device='cuda:0', grad_fn=<MseLossBackward0>)
Loss tensor(8.1270, device='cuda:0', grad_fn=<MseLossBackward0>)
Loss tensor(3.8177, device='cuda:0', grad_fn=<MseLossBackward0>)
Loss tensor(1.4131, device='cuda:0', grad_fn=<MseLossBackward0>)
Loss tensor(2.5515, device='cuda:0', grad_fn=<MseLossBackward0>)


In [10]:
# Test the model
model.eval()
testloader = torch.utils.data.DataLoader(dataset=testset, batch_size=batch_size, shuffle=False)
test_losses = []

with torch.no_grad():
    for images, labels in tqdm(testloader, desc="Testing Progress"):
        images = torch.stack(images).to(device).float()
        images = images.permute(1, 0, 2, 3, 4)
        labels = labels.to(device).float()
        with torch.amp.autocast('cuda'):
            outputs = model(images)
            loss = criterion(outputs, labels)
        test_losses.append(loss.item())
        print("Test loss", test_losses)

avg_test_loss = sum(test_losses) / len(test_losses)
print(f'Average test loss: {avg_test_loss:.4f}')

Testing Progress:   0%|          | 0/205 [00:00<?, ?it/s]

c:\Users\mirzi\miniconda3\envs\gmimodel\lib\site-packages\torch\nn\modules\loss.py:610: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Test loss [4.347095012664795]
Test loss [4.347095012664795, 3.0658912658691406]
Test loss [4.347095012664795, 3.0658912658691406, 1.1771572828292847]
Test loss [4.347095012664795, 3.0658912658691406, 1.1771572828292847, 0.06201647222042084]
Test loss [4.347095012664795, 3.0658912658691406, 1.1771572828292847, 0.06201647222042084, 3.0658912658691406]
Test loss [4.347095012664795, 3.0658912658691406, 1.1771572828292847, 0.06201647222042084, 3.0658912658691406, 0.5639539957046509]
Test loss [4.347095012664795, 3.0658912658691406, 1.1771572828292847, 0.06201647222042084, 3.0658912658691406, 0.5639539957046509, 0.1746978759765625]
Test loss [4.347095012664795, 3.0658912658691406, 1.1771572828292847, 0.06201647222042084, 3.0658912658691406, 0.5639539957046509, 0.1746978759765625, 1.1771572828292847]
Test loss [4.347095012664795, 3.0658912658691406, 1.1771572828292847, 0.06201647222042084, 3.0658912658691406, 0.5639539957046509, 0.1746978759765625, 1.1771572828292847, 7.56782865524292]
Test l